In [1]:
import pandas as pd
import seaborn as sns
import re
from bs4 import BeautifulSoup
import numpy as np

## Requirements
As we're creating a Job Vacancies App our requirements are
1. Links to the twitter Posts
2. Links to the actual websites where the job vacancies are posted
3. The job title
4. The company name
5. User who posted the job
6. Date/Time at which the job is posted
7. Spread Factor(A gradient of how many people are talking about the job) Out of 10, based on retweets, likes and followers
8. Source where the job is posted
9. Text of the Tweet
10. Hashtags used in the tweet
11. Location of the job
12. Verified or Not

In [2]:
df = pd.read_csv('Job_Tweets.csv')
df.head(10)

ID                  Timestamp             User  \
0  1211798864656769025  2019-12-30 23:58:58+00:00         SipapuNM   
1  1211797371853705220  2019-12-30 23:53:02+00:00       LorettaOD1   
2  1211795775363145728  2019-12-30 23:46:41+00:00  guajardo_celina   
3  1211793355060981767  2019-12-30 23:37:04+00:00  SteveEckert_OTD   
4  1211792689022349315  2019-12-30 23:34:25+00:00         HireLive   
5  1211792528388841473  2019-12-30 23:33:47+00:00         HireLive   
6  1211791693005099008  2019-12-30 23:30:28+00:00         HireLive   
7  1211790931772477440  2019-12-30 23:27:26+00:00         HireLive   
8  1211789520473079809  2019-12-30 23:21:50+00:00          PRGWest   
9  1211788232620412929  2019-12-30 23:16:43+00:00           GhLisa   

                                                Text  \
0  Looking for an exciting job where you can ski ...   
1  Hail in Phoenix - no way!  New opportunity at ...   
2  Rackspace is hiring! We are looking for Nation...   
3  We are #hiring Administrative Assistant in Nan...   
4  We are #hiring Family Service Counselor in Irv...   
5  We are #hiring Inside Sales Representative in ...   
6  We are #hiring Community Service Counselor in ...   
7  We are #hiring Outside Sales Representative in...   
8  We are #hiring Workday HRIS Manager  19-00454 ...   
9  We are #hiring Restaurant Manager - Hourly Man...   

                                             Hashtag  Retweets  Likes  \
0                                                NaN         0      1   
1                                            ['job']         0      0   
2  ['becomearacker', 'Rackspace', 'recruiting', '...         0      0   
3                       ['hiring', 'jobs', 'Nanuet']         0      0   
4                       ['hiring', 'jobs', 'Irvine']         0      0   
5                       ['hiring', 'jobs', 'Irvine']         0      0   
6                       ['hiring', 'jobs', 'Irvine']         0      0   
7                       ['hiring', 'jobs', 'Irvine']         0      0   
8                       ['hiring', 'jobs', 'Irvine']         0      0   
9                       ['hiring', 'jobs', 'Laurel']         0      0   

   Replies                                             Source  \
0        0  <a href="http://instagram.com" rel="nofollow">...   
1        0  <a href="http://jobvite.com" rel="nofollow">Jo...   
2        0  <a href="http://jobvite.com" rel="nofollow">Jo...   
3        0  <a href="http://www.ziprecruiter.com" rel="nof...   
4        0  <a href="http://www.ziprecruiter.com" rel="nof...   
5        0  <a href="http://www.ziprecruiter.com" rel="nof...   
6        0  <a href="http://www.ziprecruiter.com" rel="nof...   
7        0  <a href="http://www.ziprecruiter.com" rel="nof...   
8        0  <a href="http://www.ziprecruiter.com" rel="nof...   
9        0  <a href="http://www.ziprecruiter.com" rel="nof...   

                    Location  Verified_Account  Followers  Following  
0    20 miles SE of Taos, NM             False       3342        258  
1                    Arizona             False         63        129  
2                 Texas, USA             False         19         50  
3  Orange County, California             False        966       1569  
4              United States             False        983       1251  
5              United States             False        983       1251  
6              United States             False        983       1251  
7              United States             False        983       1251  
8        Southern California             False        829       1015  
9           Mississippi, USA             False         89         80

In [3]:
print(df.columns)
print(df.shape)

Index(['ID', 'Timestamp', 'User', 'Text', 'Hashtag', 'Retweets', 'Likes',
       'Replies', 'Source', 'Location', 'Verified_Account', 'Followers',
       'Following'],
      dtype='object')
(50000, 13)


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                50000 non-null  int64 
 1   Timestamp         50000 non-null  object
 2   User              50000 non-null  object
 3   Text              50000 non-null  object
 4   Hashtag           40684 non-null  object
 5   Retweets          50000 non-null  int64 
 6   Likes             50000 non-null  int64 
 7   Replies           50000 non-null  int64 
 8   Source            50000 non-null  object
 9   Location          44011 non-null  object
 10  Verified_Account  50000 non-null  bool  
 11  Followers         50000 non-null  int64 
 12  Following         50000 non-null  int64 
dtypes: bool(1), int64(6), object(6)
memory usage: 4.6+ MB


In [5]:
sns.heatmap(df.isnull())

<Axes: >

In [6]:
newdf = pd.DataFrame(
    columns=['tlink', 'wlink', 'user', 'datetime', 'spreadfactor', 'hiringchance', 'source', 'text', 'hashtags',
             'location', 'verified'])
newdf

Empty DataFrame
Columns: [tlink, wlink, user, datetime, spreadfactor, hiringchance, source, text, hashtags, location, verified]
Index: []

In [7]:
newdf['tlink'] = 'https://twitter.com/' + df['User'] + '/status/' + df['ID'].astype(str)
newdf

tlink wlink user datetime  \
0      https://twitter.com/SipapuNM/status/1211798864...   NaN  NaN      NaN   
1      https://twitter.com/LorettaOD1/status/12117973...   NaN  NaN      NaN   
2      https://twitter.com/guajardo_celina/status/121...   NaN  NaN      NaN   
3      https://twitter.com/SteveEckert_OTD/status/121...   NaN  NaN      NaN   
4      https://twitter.com/HireLive/status/1211792689...   NaN  NaN      NaN   
...                                                  ...   ...  ...      ...   
49995  https://twitter.com/babelsblessing/status/1636...   NaN  NaN      NaN   
49996  https://twitter.com/SSARecruit/status/16363065...   NaN  NaN      NaN   
49997  https://twitter.com/StaffYourDealer/status/163...   NaN  NaN      NaN   
49998  https://twitter.com/StaffYourDealer/status/163...   NaN  NaN      NaN   
49999  https://twitter.com/PragatiEmpower/status/1636...   NaN  NaN      NaN   

      spreadfactor hiringchance source text hashtags location verified  
0              NaN          NaN    NaN  NaN      NaN      NaN      NaN  
1              NaN          NaN    NaN  NaN      NaN      NaN      NaN  
2              NaN          NaN    NaN  NaN      NaN      NaN      NaN  
3              NaN          NaN    NaN  NaN      NaN      NaN      NaN  
4              NaN          NaN    NaN  NaN      NaN      NaN      NaN  
...            ...          ...    ...  ...      ...      ...      ...  
49995          NaN          NaN    NaN  NaN      NaN      NaN      NaN  
49996          NaN          NaN    NaN  NaN      NaN      NaN      NaN  
49997          NaN          NaN    NaN  NaN      NaN      NaN      NaN  
49998          NaN          NaN    NaN  NaN      NaN      NaN      NaN  
49999          NaN          NaN    NaN  NaN      NaN      NaN      NaN  

[50000 rows x 11 columns]

In [8]:
def extract_links(df, col):
    links = pd.Series(dtype=object)
    for index, row in df.iterrows():
        text = row[col]
        matches = re.findall(r'(https?://[^\s]+)|(www\.[^\s]+)', text)
        if matches:
            link = matches[0]
            links.loc[index] = link
        else:
            links.loc[index] = None
    return links


newdf['wlink'] = extract_links(df, 'Text')
newdf

tlink  \
0      https://twitter.com/SipapuNM/status/1211798864...   
1      https://twitter.com/LorettaOD1/status/12117973...   
2      https://twitter.com/guajardo_celina/status/121...   
3      https://twitter.com/SteveEckert_OTD/status/121...   
4      https://twitter.com/HireLive/status/1211792689...   
...                                                  ...   
49995  https://twitter.com/babelsblessing/status/1636...   
49996  https://twitter.com/SSARecruit/status/16363065...   
49997  https://twitter.com/StaffYourDealer/status/163...   
49998  https://twitter.com/StaffYourDealer/status/163...   
49999  https://twitter.com/PragatiEmpower/status/1636...   

                             wlink user datetime spreadfactor hiringchance  \
0      (https://t.co/imMnymMfwW, )  NaN      NaN          NaN          NaN   
1      (https://t.co/CFAKKQHeFJ, )  NaN      NaN          NaN          NaN   
2      (https://t.co/XuSMG0eSgs, )  NaN      NaN          NaN          NaN   
3      (https://t.co/3IGxFiCXUI, )  NaN      NaN          NaN          NaN   
4      (https://t.co/XAianPE9mX, )  NaN      NaN          NaN          NaN   
...                            ...  ...      ...          ...          ...   
49995  (https://t.co/WG1UTWmsHZ, )  NaN      NaN          NaN          NaN   
49996  (https://t.co/En3UQHu7GO, )  NaN      NaN          NaN          NaN   
49997  (https://t.co/r52DOaqMRS, )  NaN      NaN          NaN          NaN   
49998  (https://t.co/FsVlBkNmmd, )  NaN      NaN          NaN          NaN   
49999  (https://t.co/t0III7CMEM, )  NaN      NaN          NaN          NaN   

      source text hashtags location verified  
0        NaN  NaN      NaN      NaN      NaN  
1        NaN  NaN      NaN      NaN      NaN  
2        NaN  NaN      NaN      NaN      NaN  
3        NaN  NaN      NaN      NaN      NaN  
4        NaN  NaN      NaN      NaN      NaN  
...      ...  ...      ...      ...      ...  
49995    NaN  NaN      NaN      NaN      NaN  
49996    NaN  NaN      NaN      NaN      NaN  
49997    NaN  NaN      NaN      NaN      NaN  
49998    NaN  NaN      NaN      NaN      NaN  
49999    NaN  NaN      NaN      NaN      NaN  

[50000 rows x 11 columns]

In [9]:
newdf['user'] = df['User']
newdf['datetime'] = df['Timestamp']
newdf['text'] = df['Text']
newdf['hashtags'] = df['Hashtag']
newdf['location'] = df['Location']
newdf['verified'] = df['Verified_Account']
newdf

tlink  \
0      https://twitter.com/SipapuNM/status/1211798864...   
1      https://twitter.com/LorettaOD1/status/12117973...   
2      https://twitter.com/guajardo_celina/status/121...   
3      https://twitter.com/SteveEckert_OTD/status/121...   
4      https://twitter.com/HireLive/status/1211792689...   
...                                                  ...   
49995  https://twitter.com/babelsblessing/status/1636...   
49996  https://twitter.com/SSARecruit/status/16363065...   
49997  https://twitter.com/StaffYourDealer/status/163...   
49998  https://twitter.com/StaffYourDealer/status/163...   
49999  https://twitter.com/PragatiEmpower/status/1636...   

                             wlink             user  \
0      (https://t.co/imMnymMfwW, )         SipapuNM   
1      (https://t.co/CFAKKQHeFJ, )       LorettaOD1   
2      (https://t.co/XuSMG0eSgs, )  guajardo_celina   
3      (https://t.co/3IGxFiCXUI, )  SteveEckert_OTD   
4      (https://t.co/XAianPE9mX, )         HireLive   
...                            ...              ...   
49995  (https://t.co/WG1UTWmsHZ, )   babelsblessing   
49996  (https://t.co/En3UQHu7GO, )       SSARecruit   
49997  (https://t.co/r52DOaqMRS, )  StaffYourDealer   
49998  (https://t.co/FsVlBkNmmd, )  StaffYourDealer   
49999  (https://t.co/t0III7CMEM, )   PragatiEmpower   

                        datetime spreadfactor hiringchance source  \
0      2019-12-30 23:58:58+00:00          NaN          NaN    NaN   
1      2019-12-30 23:53:02+00:00          NaN          NaN    NaN   
2      2019-12-30 23:46:41+00:00          NaN          NaN    NaN   
3      2019-12-30 23:37:04+00:00          NaN          NaN    NaN   
4      2019-12-30 23:34:25+00:00          NaN          NaN    NaN   
...                          ...          ...          ...    ...   
49995  2023-03-16 10:04:39+00:00          NaN          NaN    NaN   
49996  2023-03-16 10:01:07+00:00          NaN          NaN    NaN   
49997  2023-03-16 09:55:14+00:00          NaN          NaN    NaN   
49998  2023-03-16 09:54:43+00:00          NaN          NaN    NaN   
49999  2023-03-16 09:54:42+00:00          NaN          NaN    NaN   

                                                    text  \
0      Looking for an exciting job where you can ski ...   
1      Hail in Phoenix - no way!  New opportunity at ...   
2      Rackspace is hiring! We are looking for Nation...   
3      We are #hiring Administrative Assistant in Nan...   
4      We are #hiring Family Service Counselor in Irv...   
...                                                  ...   
49995  We are hiring! 😍\nIf you have experience in pr...   
49996  A lovely review from one of our wonderful cand...   
49997  We are #hiring Automotive Service Advisor Writ...   
49998  We are #hiring Automotive Sales Consultant in ...   
49999  We Are Hiring!\n\nIf you think you fit in this...   

                                                hashtags  \
0                                                    NaN   
1                                                ['job']   
2      ['becomearacker', 'Rackspace', 'recruiting', '...   
3                           ['hiring', 'jobs', 'Nanuet']   
4                           ['hiring', 'jobs', 'Irvine']   
...                                                  ...   
49995           ['projectcoordinator', 'london', 'jobs']   
49996  ['SSADigital', 'Recruitment', 'Hiring', 'Review']   
49997                     ['hiring', 'jobs', 'Grayling']   
49998                     ['hiring', 'jobs', 'Grayling']   
49999  ['hiring', 'nqsf', 'recruitment', 'vocationalt...   

                             location  verified  
0             20 miles SE of Taos, NM     False  
1                             Arizona     False  
2                          Texas, USA     False  
3           Orange County, California     False  
4                       United States     False  
...                               ...       ...  
49995                 London, England     False 

In [10]:
def extract_source(df, col):
    source = pd.Series(dtype=object)
    for index, row in df.iterrows():
        element = row[col]
        soup = BeautifulSoup(element, 'html.parser')
        a = soup.find('a')
        if a:
            text = a.text.strip()
            source.loc[index] = text
        else:
            source.loc[index] = None
    return source


newdf['source'] = extract_source(df, 'Source')
newdf

tlink  \
0      https://twitter.com/SipapuNM/status/1211798864...   
1      https://twitter.com/LorettaOD1/status/12117973...   
2      https://twitter.com/guajardo_celina/status/121...   
3      https://twitter.com/SteveEckert_OTD/status/121...   
4      https://twitter.com/HireLive/status/1211792689...   
...                                                  ...   
49995  https://twitter.com/babelsblessing/status/1636...   
49996  https://twitter.com/SSARecruit/status/16363065...   
49997  https://twitter.com/StaffYourDealer/status/163...   
49998  https://twitter.com/StaffYourDealer/status/163...   
49999  https://twitter.com/PragatiEmpower/status/1636...   

                             wlink             user  \
0      (https://t.co/imMnymMfwW, )         SipapuNM   
1      (https://t.co/CFAKKQHeFJ, )       LorettaOD1   
2      (https://t.co/XuSMG0eSgs, )  guajardo_celina   
3      (https://t.co/3IGxFiCXUI, )  SteveEckert_OTD   
4      (https://t.co/XAianPE9mX, )         HireLive   
...                            ...              ...   
49995  (https://t.co/WG1UTWmsHZ, )   babelsblessing   
49996  (https://t.co/En3UQHu7GO, )       SSARecruit   
49997  (https://t.co/r52DOaqMRS, )  StaffYourDealer   
49998  (https://t.co/FsVlBkNmmd, )  StaffYourDealer   
49999  (https://t.co/t0III7CMEM, )   PragatiEmpower   

                        datetime spreadfactor hiringchance  \
0      2019-12-30 23:58:58+00:00          NaN          NaN   
1      2019-12-30 23:53:02+00:00          NaN          NaN   
2      2019-12-30 23:46:41+00:00          NaN          NaN   
3      2019-12-30 23:37:04+00:00          NaN          NaN   
4      2019-12-30 23:34:25+00:00          NaN          NaN   
...                          ...          ...          ...   
49995  2023-03-16 10:04:39+00:00          NaN          NaN   
49996  2023-03-16 10:01:07+00:00          NaN          NaN   
49997  2023-03-16 09:55:14+00:00          NaN          NaN   
49998  2023-03-16 09:54:43+00:00          NaN          NaN   
49999  2023-03-16 09:54:42+00:00          NaN          NaN   

                       source  \
0                   Instagram   
1                     Jobvite   
2                     Jobvite   
3      ZipRecruiter Post Jobs   
4      ZipRecruiter Post Jobs   
...                       ...   
49995      Twitter for iPhone   
49996                  Buffer   
49997  ZipRecruiter Post Jobs   
49998  ZipRecruiter Post Jobs   
49999         Twitter Web App   

                                                    text  \
0      Looking for an exciting job where you can ski ...   
1      Hail in Phoenix - no way!  New opportunity at ...   
2      Rackspace is hiring! We are looking for Nation...   
3      We are #hiring Administrative Assistant in Nan...   
4      We are #hiring Family Service Counselor in Irv...   
...                                                  ...   
49995  We are hiring! 😍\nIf you have experience in pr...   
49996  A lovely review from one of our wonderful cand...   
49997  We are #hiring Automotive Service Advisor Writ...   
49998  We are #hiring Automotive Sales Consultant in ...   
49999  We Are Hiring!\n\nIf you think you fit in this...   

                                                hashtags  \
0                                                    NaN   
1                                                ['job']   
2      ['becomearacker', 'Rackspace', 'recruiting', '...   
3                           ['hiring', 'jobs', 'Nanuet']   
4                           ['hiring', 'jobs', 'Irvine']   
...                                                  ...   
49995           ['projectcoordinator', 'london', 'jobs']   
49996  ['SSADigital', 'Recruitment', 'Hiring', 'Review']   
49997                     ['hiring', 'jobs', 'Grayling']   
49998                     ['hiring', 'jobs', 'Grayling']   
49999  ['hiring', 'nqsf', 'recruitment', 'vocationalt...   

                             location  verified  
0             20 miles SE of Taos, 



spreadfactor is a gradient(from 0 to 10) about how much the tweets have spread. More spreadfactor means it has spread more and the chances of hiring are less
The gradients that define it are Likes, Replies, Retweets, Followers, and Following. 
Likes, Replies, and Retweets have more weightage to the factor, while the other two have less

hiringchance is basically another gradient from 1 to 10 about the hiring chance, the more the hiringchance, the better the chances of getting hired
here the column "Time" is incorporated in as more Time goes by the Hiring Chance Decreases. 
spreadfactor is also incorporated, more the spreadfactor, less are the chances you get hired

In [11]:
def calculate_spreadfactor(row):
    weights = {"Likes": 0.35, "Replies": 0.35, "Retweets": 0.349, "Followers": 0.0005, "Following": 0.0005}
    spreadfactor = sum(row[col] * weights[col] for col in weights.keys())
    return min(max(spreadfactor, 0), 10)


def calculate_hiringchance(row):
    # print(pd.to_datetime(pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')))
    # print(pd.to_datetime(pd.to_datetime(row["datetime"]).strftime('%Y-%m-%d %H:%M:%S')))
    time_delta = (pd.to_datetime(pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')) - pd.to_datetime(
        pd.to_datetime(row["datetime"]).strftime('%Y-%m-%d %H:%M:%S'))) / pd.Timedelta(days=1)
    hiringchance = 10 - ((time_delta - 1554) * 50) - (row["spreadfactor"] * 0.2)
    # print(hiringchance)
    # return min(max(hiringchance, 1), 10)
    if hiringchance:return hiringchance
    else: return 0


x = df.apply(calculate_spreadfactor, axis=1)
x_norm = (x - np.min(x)) / (np.max(x) - np.min(x))

y = newdf.apply(calculate_hiringchance, axis=1)
y_norm = (y - np.min(y)) / (np.max(y) - np.min(y))




In [12]:
newdf["spreadfactor"] = x_norm
newdf["hiringchance"] = 2*((y_norm * 100000) - 1740)
newdf

tlink  \
0      https://twitter.com/SipapuNM/status/1211798864...   
1      https://twitter.com/LorettaOD1/status/12117973...   
2      https://twitter.com/guajardo_celina/status/121...   
3      https://twitter.com/SteveEckert_OTD/status/121...   
4      https://twitter.com/HireLive/status/1211792689...   
...                                                  ...   
49995  https://twitter.com/babelsblessing/status/1636...   
49996  https://twitter.com/SSARecruit/status/16363065...   
49997  https://twitter.com/StaffYourDealer/status/163...   
49998  https://twitter.com/StaffYourDealer/status/163...   
49999  https://twitter.com/PragatiEmpower/status/1636...   

                             wlink             user  \
0      (https://t.co/imMnymMfwW, )         SipapuNM   
1      (https://t.co/CFAKKQHeFJ, )       LorettaOD1   
2      (https://t.co/XuSMG0eSgs, )  guajardo_celina   
3      (https://t.co/3IGxFiCXUI, )  SteveEckert_OTD   
4      (https://t.co/XAianPE9mX, )         HireLive   
...                            ...              ...   
49995  (https://t.co/WG1UTWmsHZ, )   babelsblessing   
49996  (https://t.co/En3UQHu7GO, )       SSARecruit   
49997  (https://t.co/r52DOaqMRS, )  StaffYourDealer   
49998  (https://t.co/FsVlBkNmmd, )  StaffYourDealer   
49999  (https://t.co/t0III7CMEM, )   PragatiEmpower   

                        datetime  spreadfactor  hiringchance  \
0      2019-12-30 23:58:58+00:00       0.21500           NaN   
1      2019-12-30 23:53:02+00:00       0.00960           NaN   
2      2019-12-30 23:46:41+00:00       0.00345           NaN   
3      2019-12-30 23:37:04+00:00       0.12675           NaN   
4      2019-12-30 23:34:25+00:00       0.11170           NaN   
...                          ...           ...           ...   
49995  2023-03-16 10:04:39+00:00       0.09800           NaN   
49996  2023-03-16 10:01:07+00:00       0.63935           NaN   
49997  2023-03-16 09:55:14+00:00       0.12870           NaN   
49998  2023-03-16 09:54:43+00:00       0.12870           NaN   
49999  2023-03-16 09:54:42+00:00       0.14955           NaN   

                       source  \
0                   Instagram   
1                     Jobvite   
2                     Jobvite   
3      ZipRecruiter Post Jobs   
4      ZipRecruiter Post Jobs   
...                       ...   
49995      Twitter for iPhone   
49996                  Buffer   
49997  ZipRecruiter Post Jobs   
49998  ZipRecruiter Post Jobs   
49999         Twitter Web App   

                                                    text  \
0      Looking for an exciting job where you can ski ...   
1      Hail in Phoenix - no way!  New opportunity at ...   
2      Rackspace is hiring! We are looking for Nation...   
3      We are #hiring Administrative Assistant in Nan...   
4      We are #hiring Family Service Counselor in Irv...   
...                                                  ...   
49995  We are hiring! 😍\nIf you have experience in pr...   
49996  A lovely review from one of our wonderful cand...   
49997  We are #hiring Automotive Service Advisor Writ...   
49998  We are #hiring Automotive Sales Consultant in ...   
49999  We Are Hiring!\n\nIf you think you fit in this...   

                                                hashtags  \
0                                                    NaN   
1                                                ['job']   
2      ['becomearacker', 'Rackspace', 'recruiting', '...   
3                           ['hiring', 'jobs', 'Nanuet']   
4                           ['hiring', 'jobs', 'Irvine']   
...                                                  ...   
49995           ['projectcoordinator', 'london', 'jobs']   
49996  ['SSADigital', 'Recruitment', 'Hiring', 'Review']   
49997                     ['hiring', 'jobs', 'Grayling']   
49998                     ['hiring', 'jobs', 'Grayling']   
49999  ['hiring', 'nqsf', 'recruitment', 'vocationalt...   

                             location  verified  
0          

In [13]:
sorted_df = newdf.sort_values(by='datetime', ascending=False)
sorted_df

tlink  \
40000  https://twitter.com/totalcaretwg/status/164553...   
40001  https://twitter.com/ultabeautyjobs/status/1645...   
40002  https://twitter.com/EEIHR/status/1645532189965...   
40003  https://twitter.com/COHCareers/status/16455321...   
40004  https://twitter.com/nidecsvprobe/status/164553...   
...                                                  ...   
9995   https://twitter.com/FayMRecruitment/status/120...   
9996   https://twitter.com/ManpowerCoy/status/1204098...   
9997   https://twitter.com/Recruitingplus1/status/120...   
9998   https://twitter.com/SmithLatifal/status/120409...   
9999   https://twitter.com/jgadison2020/status/120409...   

                                                   wlink             user  \
40000                        (https://t.co/3X9RsvTtEx, )     totalcaretwg   
40001                        (https://t.co/r7jNUPXHgu, )   ultabeautyjobs   
40002                        (https://t.co/rT8RWlKkj3, )            EEIHR   
40003                        (https://t.co/hOkFKtAbbK, )       COHCareers   
40004                        (https://t.co/GMWLipb8QD, )     nidecsvprobe   
...                                                  ...              ...   
9995                         (https://t.co/lPkGoX2xwG, )  FayMRecruitment   
9996                         (https://t.co/XlM6iFz5Fp, )      ManpowerCoy   
9997                         (https://t.co/mwHtf5ebpx, )  Recruitingplus1   
9998   (https://t.co/CWvvZgSHOG"&gt;https://t.co/CWvv...     SmithLatifal   
9999                         (https://t.co/a44O7o0GF8, )     jgadison2020   

                        datetime  spreadfactor  hiringchance           source  \
40000  2023-04-10 21:00:55+00:00       0.00065           NaN       LaterMedia   
40001  2023-04-10 21:00:45+00:00       0.06790           NaN    CareerArc App   
40002  2023-04-10 21:00:21+00:00       0.10470           NaN   Hootsuite Inc.   
40003  2023-04-10 21:00:16+00:00       0.25410           NaN   Hootsuite Inc.   
40004  2023-04-10 20:58:34+00:00       0.01150           NaN  Twitter Web App   
...                          ...           ...           ...              ...   
9995   2019-12-09 18:01:05+00:00       0.25980           NaN   Hootsuite Inc.   
9996   2019-12-09 18:00:22+00:00       0.03995           NaN  MightyRecruiter   
9997   2019-12-09 18:00:21+00:00       0.00375           NaN  MightyRecruiter   
9998   2019-12-09 18:00:21+00:00       0.02470           NaN  MightyRecruiter   
9999   2019-12-09 18:00:20+00:00       0.00035           NaN  MightyRecruiter   

                                                    text  \
40000  WE ARE HIRING! Licensed Practical Nurse (LPN) ...   
40001  Looking to take your career to the next level ...   
40002  When job seekers apply to online applications,...   
40003  JOB OPENING! WE ARE HIRING! The HOUSTON INFORM...   
40004  We are #hiring! If you are interested in any o...   
...                                                  ...   
9995   We are hiring! Get in touch today info@faymaye...   
9996   We are hiring! Click the link below for job de...   
9997   We are hiring! Click the link below for job de...   
9998   We are hiring! Click the link below for job de...   
9999   We are hiring! Click the link below for job de...   

                                                hashtags           location  \
40000                                  ['nurse', 'jobs']                NaN   
40001              ['beautywithoutlimits', 'LuxeBeauty']                NaN   
40002                   ['employment', 'jobs', 'hiring']       Manassas, VA   
40003                                                NaN        Houston, TX   
40004   ['hiring', 'greatplacetowork', 'hiring', 'jobs']     Tempe, Arizona   
...                                                  ...                ...   
9995   ['job', 'jobs', 'hiring', 'wearehiring', 'newj...  Stafford, England   
9996                                                 NaN     Fort Worth, TX   


In [14]:
sorted_df.to_csv("Finalized-Tweets.csv", index=False)
